# Quick Start: Homodyne XPCS Analysis

This notebook demonstrates the complete homodyne workflow in 10 minutes:

1. Load an XPCS dataset from an HDF5 file
2. Configure and run NLSQ static-mode fitting
3. Visualize the results
4. Interpret the fitted parameters

**Requirements:** homodyne installed (`uv sync`), an HDF5 data file.

---

## 1. Environment Setup

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt

# Homodyne imports
import homodyne
from homodyne.config import ConfigManager
from homodyne.data import load_xpcs_data
from homodyne.optimization.nlsq import fit_nlsq_jax
from homodyne.utils.logging import get_logger

print(f"Homodyne version: {homodyne.__version__}")

# Check optimization backend availability
from homodyne.optimization import OPTIMIZATION_STATUS
print(f"NLSQ available: {OPTIMIZATION_STATUS['nlsq_available']}")
print(f"CMC available:  {OPTIMIZATION_STATUS['cmc_available']}")

logger = get_logger(__name__)

## 2. Configuration

Homodyne is driven by a YAML configuration file. You can generate a template
with the CLI:

```bash
homodyne-config --mode static --output config_quickstart.yaml
```

Here we create a minimal configuration programmatically:

In [ ]:
# Create a minimal YAML configuration string
config_yaml = """
data:
  file_path: "sample_data.h5"       # Replace with your HDF5 file path
  dataset_path: "/exchange/data"    # Internal HDF5 path to C2 array
  q_value: 0.054                    # Scattering vector in Å⁻¹
  dt: 0.1                           # Frame interval in seconds

analysis:
  mode: "static"                    # Static mode: 3 parameters

optimization:
  method: "nlsq"
  nlsq:
    anti_degeneracy:
      per_angle_mode: "auto"        # Recommended: prevents parameter absorption

parameter_space:
  D0:
    initial: 1000.0
    bounds: [0.1, 1.0e6]
  alpha:
    initial: -0.5
    bounds: [-2.0, 1.0]
  D_offset:
    initial: 0.01
    bounds: [0.0, 1.0e4]
"""

# Save the config to a temporary file
import tempfile, os
config_file = tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False)
config_file.write(config_yaml)
config_file.close()
print(f"Config written to: {config_file.name}")

In [ ]:
# Load the configuration
config = ConfigManager.from_yaml(config_file.name)

print(f"Analysis mode: {config.analysis_mode}")
print(f"Initial parameters: {config.get_initial_parameters()}")

## 3. Loading Data

The `load_xpcs_data` function reads an HDF5 file and returns a standardized
dictionary. We create synthetic data here for demonstration.

In [ ]:
# Create synthetic XPCS data for demonstration
# In a real analysis, use: data = load_xpcs_data(config_file.name)

def generate_synthetic_c2(
    q=0.054,          # Å⁻¹
    D0=1200.0,        # Å²/s
    alpha=-0.5,       # sub-diffusion
    D_offset=0.05,    # Å²/s
    beta=0.12,        # speckle contrast
    offset=1.0,       # background
    n_phi=5,          # number of angles
    n_t=40,           # time points
    dt=0.1,           # s
    noise_level=0.005,
    seed=42,
):
    """Generate synthetic two-time correlation data."""
    rng = np.random.default_rng(seed)
    t = dt * np.arange(n_t)
    phi = np.linspace(0, 180, n_phi)

    # Build C2 matrix for each angle
    c2 = np.zeros((n_phi, n_t, n_t))
    for i_phi in range(n_phi):
        for i_t1, t1 in enumerate(t):
            for i_t2, t2 in enumerate(t):
                if t2 < t1:
                    continue
                dt_lag = t2 - t1
                # Diffusion kernel
                J = (D0 * (t2**(alpha+1) - t1**(alpha+1)) / (alpha+1)
                     + D_offset * dt_lag)
                g1_sq = np.exp(-2 * q**2 * J)
                c2_val = offset + beta * g1_sq
                c2[i_phi, i_t1, i_t2] = c2_val
                c2[i_phi, i_t2, i_t1] = c2_val  # symmetrize

    # Add noise
    c2 += noise_level * rng.standard_normal(c2.shape)

    return {
        'c2_exp': c2,
        't1': t,
        't2': t,
        'phi_angles_list': phi,
        'wavevector_q_list': np.array([q]),
        'sigma': noise_level * np.ones_like(c2),
        'L': 5.0e6,    # 500 µm in Å
        'dt': dt,
    }


# Generate data with known parameters
TRUE_PARAMS = {'D0': 1200.0, 'alpha': -0.5, 'D_offset': 0.05}
data = generate_synthetic_c2(**TRUE_PARAMS)

print(f"C2 shape: {data['c2_exp'].shape}  # (n_phi, n_t1, n_t2)")
print(f"Time points: {len(data['t1'])}")
print(f"Angles: {data['phi_angles_list']}")
print(f"q = {data['wavevector_q_list'][0]:.4f} Å⁻¹")

## 4. Visualize the Raw Data

Before fitting, always inspect the raw C2 data.

In [ ]:
fig, axes = plt.subplots(1, data['c2_exp'].shape[0], figsize=(3 * data['c2_exp'].shape[0], 3))
if data['c2_exp'].shape[0] == 1:
    axes = [axes]

phi = data['phi_angles_list']
t = data['t1']
c2 = data['c2_exp']

for i_phi, ax in enumerate(axes):
    im = ax.pcolormesh(t, t, c2[i_phi].T, cmap='hot', vmin=0.98, vmax=1.12, shading='auto')
    ax.set_xlabel('t1 (s)')
    ax.set_ylabel('t2 (s)')
    ax.set_title(f'φ = {phi[i_phi]:.0f}°')
    plt.colorbar(im, ax=ax, label='C2')

plt.suptitle('Two-Time Correlation Matrix', y=1.02, fontsize=12)
plt.tight_layout()
plt.show()

print("Interpretation:")
print("  - Bright region near diagonal = fast dynamics (short lag)")
print("  - Uniform off-diagonal = long-time background (offset)")
print("  - Similar patterns for all angles = static mode appropriate")

## 5. Run NLSQ Fitting

In [ ]:
from homodyne.utils.logging import log_phase

print("Running NLSQ optimization...")
with log_phase("NLSQ"):
    result = fit_nlsq_jax(data, config)

print(f"\nConvergence: {result.convergence_status}")
print(f"Reduced chi-squared: {result.reduced_chi_squared:.4f}")
print(f"Iterations: {result.iterations}")
print(f"Time: {result.execution_time:.2f} s")

## 6. Inspect Parameters

In [ ]:
# Display fitted vs true parameters
print(f"{'Parameter':<20} {'True':>12} {'Fitted':>12} {'Error':>12} {'Rel. Error':>12}")
print('-' * 70)

param_names = ['D0', 'alpha', 'D_offset']
for i, name in enumerate(param_names):
    true_val = TRUE_PARAMS.get(name, float('nan'))
    fitted_val = result.parameters[i]
    fitted_err = result.uncertainties[i]

    if true_val != 0 and not np.isnan(true_val):
        rel_err = abs(fitted_val - true_val) / abs(true_val) * 100
        rel_str = f"{rel_err:.1f}%"
    else:
        rel_str = "N/A"

    print(f"{name:<20} {true_val:>12.4g} {fitted_val:>12.4g} {fitted_err:>12.4g} {rel_str:>12}")

print(f"\nFit quality: {result.quality_flag}")
print(f"chi^2 / dof: {result.reduced_chi_squared:.4f}  (ideal: ~1.0)")

## 7. Visualize Fit Quality

In [ ]:
# Extract anti-diagonal cuts (fixed lag time)
def get_lag_cut(c2, t, lag_idx):
    """Get C2 values along anti-diagonal at given lag index."""
    n = c2.shape[0] - lag_idx
    c2_vals = np.array([c2[k, k + lag_idx] for k in range(n)])
    t_vals = t[:n]
    return t_vals, c2_vals


fig, axes = plt.subplots(1, 2, figsize=(11, 4))

# Left: experimental data at several lag times
ax = axes[0]
lags = [1, 5, 15, 30]
colors = plt.cm.plasma(np.linspace(0.1, 0.9, len(lags)))
for lag, color in zip(lags, colors):
    t_vals, c2_vals = get_lag_cut(c2[0], t, lag)
    lag_time = t[lag] - t[0]
    ax.plot(t_vals, c2_vals, 'o-', color=color, markersize=3,
            alpha=0.8, label=f'lag = {lag_time:.2f} s')
ax.set_xlabel('t1 (s)')
ax.set_ylabel('C2')
ax.set_title('Experimental C2 (angle 0°)')
ax.legend(fontsize=8)
ax.set_ylim(0.98, 1.15)

# Right: C2 vs lag time (anti-diagonal)
ax = axes[1]
lag_indices = range(1, len(t) - 1)
lag_times = [t[k] - t[0] for k in lag_indices]
c2_mean_lag = []
for lag_idx in lag_indices:
    _, c2_vals = get_lag_cut(c2[0], t, lag_idx)
    c2_mean_lag.append(np.mean(c2_vals))

ax.semilogx(lag_times, c2_mean_lag, 'ko', markersize=4, label='Experiment')
ax.axhline(1.0, color='gray', linestyle='--', alpha=0.5, label='Baseline')
ax.set_xlabel('Lag time (s)')
ax.set_ylabel('⟨C2⟩ (averaged over t1)')
ax.set_title(f'Decorrelation curve  (chi²_nu = {result.reduced_chi_squared:.3f})')
ax.legend()

plt.tight_layout()
plt.show()

## 8. Physical Interpretation

Let's interpret the fitted parameters in physical terms.

In [ ]:
D0_fitted = result.parameters[0]
alpha_fitted = result.parameters[1]
D0_err = result.uncertainties[0]

q = data['wavevector_q_list'][0]  # Å⁻¹

# Estimate relaxation time
# For anomalous diffusion: tau ~ (q^2 D0)^(-1/alpha)
tau_q = (q**2 * D0_fitted) ** (-1 / alpha_fitted) if alpha_fitted != 0 else float('inf')

# Estimate particle size from Stokes-Einstein (water at 25°C)
kT = 4.11e-21        # J (at 25°C)
eta_water = 8.9e-4   # Pa·s
D0_m2s = D0_fitted * 1e-20  # Å²/s → m²/s
Rh_nm = kT / (6 * np.pi * eta_water * D0_m2s) * 1e9  # Rh in nm

print("Physical Interpretation")
print("=" * 40)
print(f"D0 = {D0_fitted:.0f} ± {D0_err:.0f} Å²/s")

if alpha_fitted < 0:
    print(f"alpha = {alpha_fitted:.3f} → sub-diffusion (caged/gel-like motion)")
elif alpha_fitted > 0:
    print(f"alpha = {alpha_fitted:.3f} → super-diffusion")
else:
    print(f"alpha = {alpha_fitted:.3f} → normal Brownian diffusion")

print(f"\nEstimated particle radius (Stokes-Einstein, water 25°C):")
print(f"  Rh = {Rh_nm:.1f} nm")
print(f"\nCharacteristic relaxation time at q = {q:.4f} Å⁻¹:")
print(f"  tau_q ~ {tau_q:.3f} s")

## 9. Save Results

In [ ]:
import json
from pathlib import Path

output_dir = Path("quickstart_results")
output_dir.mkdir(exist_ok=True)

result_dict = {
    "analysis_mode": config.analysis_mode,
    "convergence_status": result.convergence_status,
    "reduced_chi_squared": float(result.reduced_chi_squared),
    "parameters": {
        "D0":       {"value": float(result.parameters[0]), "uncertainty": float(result.uncertainties[0])},
        "alpha":    {"value": float(result.parameters[1]), "uncertainty": float(result.uncertainties[1])},
        "D_offset": {"value": float(result.parameters[2]), "uncertainty": float(result.uncertainties[2])},
    },
    "execution_time_s": float(result.execution_time),
}

with open(output_dir / "nlsq_result.json", "w") as f:
    json.dump(result_dict, f, indent=2)

import numpy as np
np.savez(
    output_dir / "nlsq_arrays.npz",
    parameters=result.parameters,
    uncertainties=result.uncertainties,
    covariance=result.covariance,
)

print(f"Results saved to {output_dir}/")
print(f"  nlsq_result.json  — parameter estimates and quality metrics")
print(f"  nlsq_arrays.npz   — numerical arrays (covariance matrix, etc.)")

## 10. Next Steps

You have completed the quickstart! From here you can:

- **Replace the synthetic data** with your real HDF5 file by setting `file_path` in the config
- **Try laminar flow mode** if your sample is in a shear cell: `mode: "laminar_flow"`
- **Run Bayesian analysis** for publication-quality uncertainties: see `02_static_analysis.ipynb`
- **Explore the full config** with `homodyne-config --mode static --output my_config.yaml`

### Quick reference

| Command | Purpose |
|---------|----------|
| `homodyne --config config.yaml` | Run full analysis |
| `homodyne-config --mode static` | Generate config template |
| `homodyne-config --validate --input config.yaml` | Check config |
| `homodyne-config-xla --show` | Show CPU tuning recommendations |

In [ ]:
# Cleanup temporary config file
os.unlink(config_file.name)